In [ ]:
import xarray as xr
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import xlrd
import pyisotopomer

import bokeh
import bokeh.io
import bokeh.plotting

In [ ]:
# Setting up Bokeh to run inline

bokeh.io.output_notebook()

In [ ]:
# Setting how many lines of Pandas output to display
# Change this to scroll through all of your data

pd.set_option('display.max_rows', 50)

# Inputs

## <span style="color:orange">INPUT: Isodat files</span>
Paths to all Isodat output files you want to analyze with a common set of standards. Noah recommends using 5-day chunks.

In [ ]:
input_list = ['raw_benguela_data/240930_V_Benguela24_St2.xls',
             'raw_benguela_data/241010_V_Benguela24_St2_rerun_St3_St4_pt1.xls',
             'raw_benguela_data/241011_V_Benguela24_St4_pt2.xls',
             'raw_benguela_data/241017_V_Benguela24_St4_7.xls',]

## <span style="color:orange">INPUT: Output file names</span>
Set where you want pyisotopomer and final processed output to be saved. Keeping `None` for the pyisotopomer output locations will save with the default name in your working directory.

In [ ]:
chunkname= "240930_241017_Benguela24"

scrambling_output_file="pyisotopomer_output/scrambling_" + chunkname + ".xlsx"
isotopomers_output_file="pyisotopomer_output/isotopomers_" + chunkname + ".xlsx"
processed_output_file="processed_benguela_data/" + chunkname + ".csv"
processed_std_file = "processed_benguela_std/" + chunkname + ".csv"

## <span style="color:orange">INPUT: Size series and scrambling </span>
Path to that month's scrambling and size correction output. These are found in the Google Drive in `Project Folders/A01 standards and troubleshooting/Scrambling runs` and `Project Folders/A01 standards and troubleshooting/Size series & sensitivity`.

In [ ]:
sizeseries_excel = 'scrambling_sizecorr/240918 N2O Size Correction and Sensitivity.xlsx'
scrambling_excel = 'scrambling_sizecorr/240918 Scrambling.xlsx'

Make sure these values match what's in the spreadsheet. If the cells in the template got shifted, you'll have to adjust the next couple cells.

In [ ]:
sizeseries_df = pd.read_excel(sizeseries_excel, usecols="L:R", skiprows=49, nrows=2)
sizeseries_df

In [ ]:
# getting size series slope values

sizecorr_slope45R = sizeseries_df["45R"][0]
sizecorr_slope46R = sizeseries_df["46R"][0]
sizecorr_slope31R = sizeseries_df["31R"][0]

print("45R slope: " + str(sizecorr_slope45R))
print("46R slope: " + str(sizecorr_slope46R))
print("31R slope: " + str(sizecorr_slope31R))


In [ ]:
scrambling_df = pd.read_excel(scrambling_excel, usecols="AL:AQ", skiprows=1)
scrambling_df[0:3] #gamma and kappa in all rows should be the same, but feel free to check more

In [ ]:
gamma_initial_guess = scrambling_df["gamma"][0]
kappa_initial_guess = scrambling_df["kappa"][0]

print("gamma initial guess: " + str(gamma_initial_guess))
print("kappa initial guess: " + str(kappa_initial_guess))

## Concentration constants

In [ ]:
conc_slopes_df = pd.read_excel(sizeseries_excel, usecols="F:I", skiprows=41, nrows=5)
conc_slopes_df

In [ ]:
conc_slope = conc_slopes_df.iloc[0,1]
conc_slope_err = conc_slopes_df.iloc[1,1]

conc_int = conc_slopes_df.iloc[0,2]
conc_int_err = conc_slopes_df.iloc[1,2]

print("slope (nmol/Vs): " + str(conc_slope))
print("slope error (nmol/Vs): " + str(conc_slope_err))
print("intercept (nmol): " + str(conc_int))
print("intercept error (nmol): " + str(conc_int_err))

## Reference N$_2$O tank values
These stay constant unless the reference tank gets changed, which won't happen for a while.

In [ ]:
N2O_ref31R = 0.00373376282567055
N2O_ref45R = 0.00774102494962263
N2O_ref46R = 0.00210129522157562

## Reference values for standard gases

Currently includes S2, AESW, and 1000 ppm. Add others if you need them.

In [ ]:
### Constants
# Consider running this elsewhere and pulling these values in from a CSV

# Known values for standards

std_const = pd.DataFrame(index=["S2", "AESW", "1000ppm"], columns=["ref_tag", "d15Na", "d15Nb", "d18O"])
std_const["ref_tag"] = std_const.index

std_const.at['S2', 'd15Na'], std_const.at['S2', 'd15Nb'], std_const.at['S2', 'd18O'] = 5.55, -12.87, 32.73
std_const.at['AESW', 'd15Na'], std_const.at['AESW', 'd15Nb'], std_const.at['AESW', 'd18O'] = 15.6, -2.3, 44.4
std_const.at['1000ppm', 'd15Na'], std_const.at['1000ppm', 'd15Nb'], std_const.at['1000ppm', 'd18O'] = -0.11, -0.45, 37.83

# Derived values

std_const['d15Nbulk'] = std_const[['d15Na', 'd15Nb']].mean(axis=1)
std_const['SP'] = std_const['d15Na'] - std_const['d15Nb']

std_const['15Ralpha'] = (std_const['d15Na']/1000 + 1)*0.0036765
std_const['15Rbeta'] = (std_const['d15Nb']/1000 + 1)*0.0036765
std_const['18R'] = (std_const['d18O']/1000 + 1)*0.0020052
std_const['17R'] = (std_const['d18O']/1000 + 1)**0.516 * 0.0003799
std_const['45R'] = std_const['15Ralpha'] + std_const['15Rbeta'] + std_const['17R']
std_const['46R'] = ((std_const['15Ralpha'] + std_const['15Rbeta'])*std_const['17R'] 
                    + std_const['18R'] + std_const['15Ralpha']*std_const['15Rbeta'])

std_const['45R/45R'] = std_const['45R']/N2O_ref45R
std_const['46R/46R'] = std_const['46R']/N2O_ref46R

std_const

# Preprocessing
## Reading in Isodat data

Read the data into a Pandas dataframe and concatenate.

In [ ]:
df_sample = pd.DataFrame()
df_std = pd.DataFrame()

for filename in input_list:
    single_df_sample = pd.read_excel(filename, sheet_name="160hdspcN2OrR314546sample_V3_15")
    single_df_std = pd.read_excel(filename, sheet_name="160hdspcN2OrR314546standard_V3_")

    date = filename.split('/')[1][:6]

    single_df_sample.insert(0, 'run_date', date)
    single_df_std.insert(0, 'run_date', date)

    single_df_sample["Isodat output filename"] = filename.split('/')[-1]
    single_df_std["Isodat output filename"] = filename.split('/')[-1]
    
    # The following pulls the run date from the file name. If you used a different naming convention, 
    # forgot to change the date that day, etc., fix this manually.

    df_sample = pd.concat([single_df_sample, df_sample])
    df_std = pd.concat([single_df_std, df_std])

## Bringing data in line
This gets rid of data from CO$_2$ peaks and lines up the Area 44 and Area 30 rows for each bottle. Check the output to make sure it's working properly.

In [ ]:
def bring_in_line(df_input):

    # Adjust this to also double check peak areas?

    df_input["Unique ID"] = df_input["Identifier 1"] + '_' + df_input["run_date"]

    grouped = df_input.groupby("Time Code")

    df_new = pd.DataFrame()
    
    for group_name, df_group in grouped:

        sample_ID = df_group["Unique ID"].iloc[0]
        
        df_group = df_group[~(df_group["d 15N/14N"] > 500)]
    
        if len(df_group) == 2:
            df1 = df_group.iloc[::2].reset_index()
            df2 = df_group.iloc[1::2].reset_index()
            df = df1.assign(**{"Area 30": df2["Area 30"], "rR 31NO/30NO": df2["rR 31NO/30NO"]})

            df_new = pd.concat([df_new, df])
    
        else:
            print("Warning: only CO2 peak(s) detected for row " + str(sample_ID) + ". Row will be dropped.")
    
    df_new = df_new.set_index("Unique ID", drop=False)
    
    if not df_new.index.is_unique:
        print(df_new.index)
        raise IndexError(
            """
            Sample indices are not unique - check that the dates in your file names are unique
            and that standards from the same day have different names (e.g. 1000ppm_C).
            """)
        
    df_new = df_new.drop(columns=["index"])

    return df_new
        

In [ ]:
df_sample = bring_in_line(df_sample)
df_std = bring_in_line(df_std)

In [ ]:
df_sample

In [ ]:
df_std = df_std.drop("1000ppm_rerun_240930")
df_std

## Setting up analysis dataframe

In [ ]:
df_analysis = df_sample.copy()
df_analysis = df_analysis.rename(columns={"rR 45N2O/44N2O": "rR 45N2O/44N2O sam",
                           "rR 46N2O/44N2O": "rR 46N2O/44N2O sam",
                           "rR 31NO/30NO": "rR 31NO/30NO sam"})

# Getting values from standard sheet
df_analysis["rR 45N2O/44N2O std"] = df_std["rR 45N2O/44N2O"].values
df_analysis["rR 46N2O/44N2O std"] = df_std["rR 46N2O/44N2O"].values
df_analysis["rR 31NO/30NO std"] = df_std["rR 31NO/30NO"].values

In [ ]:
# Adding standard tags

df_analysis.insert(1, 'ref_tag', "")
df_analysis.loc[df_analysis.index.str.contains("S2_"), "ref_tag"] = "S2"
df_analysis.loc[df_analysis.index.str.contains("AESW"), "ref_tag"] = "AESW"
df_analysis.loc[df_analysis.index.str.contains("1000ppm"), "ref_tag"] = "1000ppm"

# Making sure the 1000 ppm reruns don't get counted
df_analysis.loc[df_analysis.index.str.contains("1000ppm_rerun"), "ref_tag"] = ""

# Fixing rerun duplicates (if necessary)

In [ ]:
#pd.set_option('display.max_rows', 100)
# df_analysis

In [ ]:
# Adding the area values, then keeping the higher-signal row for isotopes

# df_analysis.loc['Benguela24_0008_241010', 'Area 44'] = (
#     df_analysis.loc['Benguela24_0008_240930']["Area 44"] + df_analysis.loc['Benguela24_0008_241010']["Area 44"])

# df_analysis.loc['Benguela24_0008_241010', 'Area 30'] = (
#     df_analysis.loc['Benguela24_0008_240930']["Area 30"] + df_analysis.loc['Benguela24_0008_241010']["Area 30"])
    
# df_analysis = df_analysis.drop('Benguela24_0008_240930')

# N$_2$O concentration calculation

In [ ]:
# As written below this assumes a constant volume per sample

df_analysis['Volume'] = 0.113 #L, from Meléa's analysis
df_analysis['Volume_err'] = 0.0029 #L, from Meléa's analysis
df_analysis['N2O_nmol'] = conc_slope*df_analysis['Area 44'] + conc_int
df_analysis['N2O_nmol_err'] = conc_slope_err*df_analysis['Area 44'] + conc_int_err #CHECK THIS

df_analysis['N2O_nM'] = df_analysis['N2O_nmol']/df_analysis['Volume']
df_analysis['N2O_nM_err'] = (df_analysis['N2O_nmol_err']/df_analysis['N2O_nmol'] + 
                             df_analysis['Volume_err']/df_analysis['Volume'])*df_analysis['N2O_nM']
df_analysis


# Size correction

In [ ]:
# Raw sample/standard calculations

df_analysis["raw 45rR/45rR"] = df_analysis["rR 45N2O/44N2O sam"]/df_analysis["rR 45N2O/44N2O std"]
df_analysis["raw 46rR/46rR"] = df_analysis["rR 46N2O/44N2O sam"]/df_analysis["rR 46N2O/44N2O std"]
df_analysis["raw 31rR/31rR"] = df_analysis["rR 31NO/30NO sam"]/df_analysis["rR 31NO/30NO std"]

# Size correction - corrected to m/z=44 peak area of 20 Vs

df_analysis["size corrected 45rR/45rR"] = (sizecorr_slope45R * (20 - df_analysis["Area 44"]) 
                                           + df_analysis["raw 45rR/45rR"])
df_analysis["size corrected 46rR/46rR"] = (sizecorr_slope46R * (20 - df_analysis["Area 44"]) 
                                           + df_analysis["raw 46rR/46rR"])
df_analysis["size corrected 31rR/31rR"] = (sizecorr_slope31R * (20 - df_analysis["Area 44"]) 
                                           + df_analysis["raw 31rR/31rR"])

In [ ]:
df_analysis

# Scale normalization

In [ ]:
std_const

In [ ]:
# Selecting standards

scale_norm = pd.DataFrame()

scale_norm["ref_tag"] = df_analysis["ref_tag"]

scale_norm["Measured 45R/45R"] = df_analysis['size corrected 45rR/45rR'].where(
    scale_norm["ref_tag"].isin(["S2", "AESW", "1000ppm"]))

scale_norm["Measured 46R/46R"] = df_analysis['size corrected 46rR/46rR'].where(
    scale_norm["ref_tag"].isin(["S2", "AESW", "1000ppm"]))

scale_norm = scale_norm.dropna()

In [ ]:
# Populating with comparison values

scale_norm.loc[scale_norm["ref_tag"] == "S2", "Known 45R/45R"] = std_const.at["S2", "45R/45R"]
scale_norm.loc[scale_norm["ref_tag"] == "AESW", "Known 45R/45R"] = std_const.at["AESW", "45R/45R"]
scale_norm.loc[scale_norm["ref_tag"] == "1000ppm", "Known 45R/45R"] = std_const.at["1000ppm", "45R/45R"]

scale_norm.loc[scale_norm["ref_tag"] == "S2", "Known 46R/46R"] = std_const.at["S2", "46R/46R"]
scale_norm.loc[scale_norm["ref_tag"] == "AESW", "Known 46R/46R"] = std_const.at["AESW", "46R/46R"]
scale_norm.loc[scale_norm["ref_tag"] == "1000ppm", "Known 46R/46R"] = std_const.at["1000ppm", "46R/46R"]

scale_norm["ln(measured 45R/45R)"] = np.log(scale_norm["Measured 45R/45R"])
scale_norm["ln(measured 46R/46R)"] = np.log(scale_norm["Measured 46R/46R"])
scale_norm["ln(known 45R/45R)"] = np.log(scale_norm["Known 45R/45R"])
scale_norm["ln(known 46R/46R)"] = np.log(scale_norm["Known 46R/46R"])

In [ ]:
scale_norm

## <span style="color:orange">INPUT: Check plots and remove outlier values</span>
### List of standards to drop throughout

In [ ]:
drop_standards = []
#drop_standards=["1000ppm_1_241010", "S2_1_241010", "AESW_0911_1_241010"]

scale_norm = scale_norm.drop(drop_standards)

## 31R plot

In [ ]:
fit45 = np.polyfit(scale_norm["ln(measured 45R/45R)"], scale_norm["ln(known 45R/45R)"], 1, full=True)
m45 = fit45[0][0]
b45 = fit45[0][1]
err45 = fit45[1][0] # error to plot here?

print("45R slope: " + str(m45) + "\n45R intercept: " + str(b45))

### 45R plot

In [ ]:
fit45 = np.polyfit(scale_norm["ln(measured 45R/45R)"], scale_norm["ln(known 45R/45R)"], 1, full=True)
m45 = fit45[0][0]
b45 = fit45[0][1]
err45 = fit45[1][0] # error to plot here?

print("45R slope: " + str(m45) + "\n45R intercept: " + str(b45))

In [ ]:
p = bokeh.plotting.figure(
    max_width=500,
    height=300,
    title="45R/45R Scale Normalization",
    x_axis_label="ln(measured 45R/45R)",
    y_axis_label="ln(known 45R/45R)"
)

hvr = bokeh.models.HoverTool(tooltips="@{Unique ID}")
p.add_tools(hvr)

scatter = p.scatter("ln(measured 45R/45R)", "ln(known 45R/45R)", size=10, source=scale_norm)
hvr.renderers = [scatter]

x_vals = np.linspace(scale_norm["ln(measured 45R/45R)"].min(), scale_norm["ln(measured 45R/45R)"].max())
bestfit = p.line(x_vals, np.poly1d([m45,b45])(x_vals), legend_label=f"y = {m45:.4f}x + {b45:.4f}")

p.legend.location = (20, 160)

bokeh.plotting.show(p)

### 46R plot

In [ ]:
fit46 = np.polyfit(scale_norm["ln(measured 46R/46R)"], scale_norm["ln(known 46R/46R)"], 1, full=True)
m46 = fit46[0][0]
b46 = fit46[0][1]
err46 = fit46[1][0]

print("46R slope: " + str(m46) + "\n46R intercept: " + str(b46))

In [ ]:
p = bokeh.plotting.figure(
    max_width=500,
    height=300,
    title="46R/46R Scale Normalization",
    x_axis_label="ln(measured 46R/46R)",
    y_axis_label="ln(known 46R/46R)",
)

hvr = bokeh.models.HoverTool(tooltips="@{Unique ID}")
p.add_tools(hvr)

scatter = p.scatter("ln(measured 46R/46R)", "ln(known 46R/46R)", size=10, source=scale_norm)
hvr.renderers = [scatter]

x_vals = np.linspace(scale_norm["ln(measured 46R/46R)"].min(), scale_norm["ln(measured 46R/46R)"].max())
bestfit = p.line(x_vals, np.poly1d([m46,b46])(x_vals), legend_label=f"y = {m46:.4f}x + {b46:.4f}")
p.legend.location = (20, 160)

bokeh.plotting.show(p)

## Apply scale normalization 

In [ ]:
df_analysis["scale decompressed 45rR/45rR"] = df_analysis["size corrected 45rR/45rR"]**m45 * np.exp(b45)
df_analysis["scale decompressed 46rR/46rR"] = df_analysis["size corrected 46rR/46rR"]**m46 * np.exp(b46)

## Multiply by R values from reference N2O tank

In [ ]:
df_analysis["size corrected 31R"] = df_analysis["size corrected 31rR/31rR"]*N2O_ref31R
df_analysis["size corrected 45R"] = df_analysis["scale decompressed 45rR/45rR"]*N2O_ref45R
df_analysis["size corrected 46R"] = df_analysis["scale decompressed 46rR/46rR"]*N2O_ref46R

# Interface with pyisotopomer

In [ ]:
df_analysis["D17O"] = 0. # Replace with known values if you have them, but 0 is fine

In [ ]:
# This is from the earlier scale normalization step - make sure it's accurate

print(drop_standards)

In [ ]:
df_analysis = df_analysis.drop(drop_standards)
df_analysis

## Run scrambling

In [ ]:
# Add the kwarg outputfile="filename.csv" if you want to control save location,
# or saveout=False if you don't want to save a file.

scrambling_output = pyisotopomer.Scrambling(inputdf=df_analysis, refdf=std_const, method="least_squares",
                                            initialguess=[gamma_initial_guess, kappa_initial_guess],
                                           outputfile=scrambling_output_file).alloutputs

In [ ]:
scrambling_output

In [ ]:
gamma_mean = scrambling_output['gamma'].mean()
kappa_mean = scrambling_output['kappa'].mean()

print("gamma: " + str(gamma_mean) + "\nkappa: " + str(kappa_mean))

In [ ]:
df_analysis['gamma'] = gamma_mean
df_analysis['kappa'] = kappa_mean

## Run isotopomers

In [ ]:
# Add the kwarg outputfile="filename.csv" if you want to control save location,
# or saveout=False if you don't want to save a file.

isotopomersoutput = pyisotopomer.Isotopomers(inputdf=df_analysis, outputfile=isotopomers_output_file).deltavals

In [ ]:
isotopomersoutput

In [ ]:
# Reorganizing

df_output = isotopomersoutput.copy()
df_output["Unique ID"] = df_analysis["Unique ID"].values
df_output= df_output.set_index("Unique ID")

In [ ]:
# Grabbing any extra columns you want from df_analysis - add others if desired

cols_to_add = ['Isodat output filename', 'N2O_nmol', 'N2O_nmol_err', 'N2O_nM', 'N2O_nM_err']

df_output = pd.merge(df_output, df_analysis[cols_to_add], on="Unique ID")

# Rearranging order

df_output.insert(0, 'Isodat output filename', df_output.pop('Isodat output filename'))

In [ ]:
# Dropping rows with standards

df_std_output = df_output[df_output.index.str.contains("S2_|AESW|1000ppm")]
df_output = df_output[~df_output.index.str.contains("S2_|AESW|1000ppm")]


In [ ]:
df_std_output

## <span style="color:orange">INPUT: Check standard isotopomers</span>

If you want to remove any points, add those standards to `drop_standards` and rerun the pipeline

In [ ]:
aesw = df_std_output[df_std_output["Identifier 1"].str.contains("AESW")]

ppm1000 = df_std_output[df_std_output["Identifier 1"].str.contains("1000ppm")]
ppm1000 = ppm1000[~ppm1000["Identifier 1"].str.contains("rerun")]

s2 = df_std_output[df_std_output["Identifier 1"].str.contains("S2")]

In [ ]:
aesw

In [ ]:
fig, axs = plt.subplots(1, 3, figsize=(12, 5))

# AESW

axs[0].hist(aesw.SP, bins=5, color='lightblue', edgecolor='white')

min_y, max_y = axs[0].get_ylim()

axs[0].vlines(17.9, ymin=min_y, ymax=max_y, color='r', label="expected")
axs[0].vlines(aesw.SP.mean(), ymin=min_y, ymax=max_y, color='k', label="mean")

aesw_stdev = aesw.SP.std()
axs[0].vlines(aesw.SP.mean() - aesw_stdev, ymin=min_y, ymax=max_y, color='grey', ls='--', label=" 1 stdev")
axs[0].vlines(aesw.SP.mean() + aesw_stdev, ymin=min_y, ymax=max_y, color='grey', ls='--')
axs[0].set_title("AESW")
axs[0].set_ylabel("Counts")

# S2

axs[1].hist(s2.SP, bins=5, color='lightgreen', edgecolor='white')

min_y, max_y = axs[1].get_ylim()

axs[1].vlines(18.42, ymin=min_y, ymax=max_y, color='r', label="expected")
axs[1].vlines(s2.SP.mean(), ymin=min_y, ymax=max_y, color='k', label="mean")

s2_stdev = s2.SP.std()
axs[1].vlines(s2.SP.mean() - s2_stdev, ymin=min_y, ymax=max_y, color='grey', ls='--', label=" 1 stdev")
axs[1].vlines(s2.SP.mean() + s2_stdev, ymin=min_y, ymax=max_y, color='grey', ls='--')
axs[1].set_title("S2")

# 1000 ppm

axs[2].hist(ppm1000.SP, bins=5, color='sandybrown', edgecolor='white')

min_y, max_y = axs[2].get_ylim()

axs[2].vlines(0.34, ymin=min_y, ymax=max_y, color='r', label="expected")
axs[2].vlines(ppm1000.SP.mean(), ymin=min_y, ymax=max_y, color='k', label="mean")

ppm1000_stdev = ppm1000.SP.std()
axs[2].vlines(ppm1000.SP.mean() - ppm1000_stdev, ymin=min_y, ymax=max_y, color='grey', ls='--', label=" 1 stdev")
axs[2].vlines(ppm1000.SP.mean() + ppm1000_stdev, ymin=min_y, ymax=max_y, color='grey', ls='--')
axs[2].set_title("1000 ppm")

for ax in axs:
    ax.legend()
    ax.set_xlabel("Site preference")

plt.show()

# Make a nice final file with just your data
Includes original file name, delta and site preference values, and N2O concentrations.

In [ ]:
# Uncomment to see all rows

#pd.set_option('display.max_rows', None)

df_output

In [ ]:
# Writing to file

df_output.to_csv(processed_output_file)

# If you also want to write out your standards separately 

#df_std_output.to_csv(processed_std_file)

#df_output.to_excel("test.xlsx")